In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
import warnings

data = pd.read_csv('../data/ERP_data.csv')
labels = list(data.columns.values)
del labels[0]
del labels[0]

targets = data['Phenotype']
del data['Subject']
del data['Phenotype']

In [2]:
from sklearn import preprocessing, feature_selection, cross_validation, ensemble

folds = 10

imp = preprocessing.Imputer()
data = imp.fit_transform(data, targets)
data = preprocessing.scale(data)
data = feature_selection.SelectKBest(feature_selection.f_classif, k=11).fit_transform(data, targets)

# data = np.asarray(pd.DataFrame(data, columns=labels)[['P3aDACz', 'P200DAF3', 'P200DAF4', 'P3aDAvgACz',
             # 'P3aDAvgAF3', 'P3aDAP4', 'P3aDAvgAFz', 'P200DAvgAF4', 'P3aDAvgAF4']])

In [3]:
from sklearn import metrics

clf = ensemble.RandomForestClassifier(n_estimators=20, min_samples_split=3, min_samples_leaf=2)

accuracies = []
precisions = []
recalls = []
fscores = []
confusions = []
warnings.filterwarnings("ignore")
for i in range(200):
    cross_val = cross_validation.StratifiedKFold(targets, n_folds=folds, shuffle=True)
    preds = cross_validation.cross_val_predict(clf, data, targets, cv=cross_val)
    accuracy = metrics.accuracy_score(targets, preds)
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(
        targets, preds, average='binary', pos_label='AD')
    confusion = metrics.confusion_matrix(targets, preds, labels=['HC', 'AD'])
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    confusions.append(confusion)

print("Accuracy: {0}, with std: {1}".format(np.mean(accuracies), np.std(accuracies)))
print("Precision: {0}".format(np.mean(precisions)))
print("Recall: {0}".format(np.mean(recalls)))
print("F1 Score: {0}".format(np.mean(fscores)))
print("Confusion Matrix:\n   HC     AD\n{0}".format(np.mean(confusions, axis=0)))

Accuracy: 0.6864070351758795, with std: 0.01827168778135821
Precision: 0.6759476734407349
Recall: 0.710959595959596
F1 Score: 0.6927937421975778
Confusion Matrix:
   HC     AD
[[ 66.21   33.79 ]
 [ 28.615  70.385]]
